In [18]:
import os
os.chdir("C:/Users/AHMET/PycharmProjects/oae_unsw")

import sys
sys.path.append("C:/Users/AHMET/PycharmProjects/oae_unsw")

import pandas as pd
import numpy as np
import random
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline
import ydata_profiling
from tqdm import tqdm
from sklearn.metrics import roc_auc_score,confusion_matrix,accuracy_score,classification_report,roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
import sklearn
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader

from utils.autoencoder import autoencoder
from utils.Data_gen import Data_gen
from utils.get_result import get_result

np.random.seed(123)
random.seed(123)

train_data_df = pd.read_csv("./data/unsw/UNSW_NB15_training-set.csv")
test_data_df = pd.read_csv("./data/unsw/UNSW_NB15_testing-set.csv")

##label 0:n, 1:an
data_df=pd.concat([train_data_df,test_data_df],axis=0)
data_df=data_df.drop(["id"],axis=1)
data_df=data_df.drop(["attack_cat"],axis=1)

cat_vars=["proto","service","state","is_ftp_login","is_sm_ips_ports"]
cat_data = pd.get_dummies(data_df[cat_vars])

numeric_vars = list(set(data_df.columns.values.tolist()) - set(cat_vars))
numeric_vars.remove('label')
numeric_data = data_df[numeric_vars].copy()

label_data = data_df['label']

final_data_df=pd.concat([numeric_data, cat_data, label_data], axis=1).reset_index(drop=True)

In [19]:
labels = final_data_df['label'].copy()
num_data = len(labels)
num_normal = np.sum(labels ==0)
normal_ratio = num_normal/num_data
print('normal_ratio(%)', normal_ratio*100, num_normal)

def reduce_data(df, ano_cnt):
    #num_anomalies = int(num_normal * (anomal_ratio /(1-anomal_ratio)))
    num_anomalies =ano_cnt
    print('number of anomalies = ', num_anomalies, 'number of normal = ', num_normal)
    anomal_labels = labels[labels !=0]
    anomal_idx = np.random.choice(anomal_labels.index, size = num_anomalies, replace = False)
    anomal_data = df.iloc[anomal_idx].copy()
    normal_data = df[labels ==0].copy()
    print('anomal_data shape = ', anomal_data.shape)
    print('normal_data shape = ',normal_data.shape)
    return pd.concat([normal_data, anomal_data], axis = 0).reset_index(drop=True)

ano_cnt=5000
final_data_reduce_df = reduce_data(final_data_df, ano_cnt)
print('reduced_data shape = ', final_data_reduce_df.shape)

input_size=len(final_data_reduce_df.columns)-1


test_an_df=final_data_reduce_df.loc[final_data_reduce_df["label"]==1].reset_index(drop=True)
n_df=final_data_reduce_df.loc[final_data_reduce_df["label"]==0].reset_index(drop=True)
n_df=sklearn.utils.shuffle(n_df).reset_index(drop=True)
test_n_df=n_df.iloc[-ano_cnt:].reset_index(drop=True)
test_df=pd.concat([test_an_df,test_n_df],axis=0).reset_index(drop=True)
train_df=n_df.iloc[:-ano_cnt].reset_index(drop=True)

min_max_scaler = preprocessing.MinMaxScaler()
train_df = min_max_scaler.fit_transform(train_df)
test_df = min_max_scaler.transform(test_df)

train_df = pd.DataFrame(train_df)
test_df = pd.DataFrame(test_df)

train_label=train_df[192]
train_df=train_df.drop([192],axis=1)

y_test=test_df[192]
x_test=test_df.drop([192],axis=1)


x_train, x_val, y_train, y_val = train_test_split(train_df,
                                                    train_label,
                                                    test_size=0.2, 
                                                    random_state=1029)

normal_ratio(%) 67.47558251941732 20242
number of anomalies =  5000 number of normal =  20242
anomal_data shape =  (5000, 193)
normal_data shape =  (20242, 193)
reduced_data shape =  (25242, 193)


In [16]:
import torch
from torch.utils.data import DataLoader
from utils.Data_gen import Data_gen  # utils klasöründeki Data_gen sınıfını içe aktarıyoruz
from utils.autoencoder import autoencoder  # utils klasöründen autoencoder
import numpy as np

# Dataset'leri oluştur
x_train_dataset = Data_gen(x_train)
x_val_dataset = Data_gen(x_val)
x_test_dataset = Data_gen(x_test)

# Dataloader'ları tanımla
batch_size = 64  # Batch boyutunu belirleyin
train_dataloader = DataLoader(x_train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = DataLoader(x_val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_dataloader = DataLoader(x_test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Modeli eğitme kısmı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan cihaz: {device}")

model = autoencoder(input_size).to(device)
criterion = torch.nn.MSELoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

save_model_path = 'ae_model.pth'
FULL_TRAIN = True
num_epochs = 100

if FULL_TRAIN:
    for epoch in range(num_epochs):
        running_training_loss = 0.0
        running_val_loss = 0.0

        # Eğitim döngüsü
        for i_batch, sample_batched in enumerate(train_dataloader):
            sample_batched = sample_batched.to(device)
            en_out, de_out, latant_v, output = model(sample_batched)
            train_loss = criterion(output, sample_batched)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            running_training_loss += train_loss.item()

        # Doğrulama döngüsü
        if epoch % 1 == 0:
            for i_batch, sample_batched in enumerate(val_dataloader):
                sample_batched = sample_batched.to(device)
                en_out, de_out, latant_v, val_output = model(sample_batched)
                val_loss = criterion(val_output, sample_batched)
                running_val_loss += val_loss.item()
            print(">>>> @ {}, training_loss = {:.4f}, val_loss = {:.4f}".format(
                epoch + 1,
                running_training_loss / len(train_dataloader),
                running_val_loss / len(val_dataloader)
            ))

        # Modeli kaydet
        torch.save(model.state_dict(), save_model_path)
else:
    print("Önceden eğitilmiş modeli yüklüyor")
    model.load_state_dict(torch.load(save_model_path))

# Test ve Eğitim Verilerinde Çalıştırma
# Test verilerini işleyin
x_test_array = x_test.values
x_test_torch = torch.from_numpy(x_test_array).type(torch.FloatTensor).to(device)
test_en_out, test_de_out, test_latant_v, x_test_recon = model(x_test_torch)

# Eğitim verilerini işleyin
x_train_array = x_train.values
x_train_torch = torch.from_numpy(x_train_array).type(torch.FloatTensor).to(device)
train_en_out, train_de_out, train_latant_v, x_train_recon = model(x_train_torch)


Kullanılan cihaz: cpu
>>>> @ 1, training_loss = 0.0051, val_loss = 0.0033
>>>> @ 2, training_loss = 0.0028, val_loss = 0.0023
>>>> @ 3, training_loss = 0.0021, val_loss = 0.0018
>>>> @ 4, training_loss = 0.0016, val_loss = 0.0013
>>>> @ 5, training_loss = 0.0013, val_loss = 0.0011
>>>> @ 6, training_loss = 0.0009, val_loss = 0.0009
>>>> @ 7, training_loss = 0.0033, val_loss = 0.0039
>>>> @ 8, training_loss = 0.0018, val_loss = 0.0021
>>>> @ 9, training_loss = 0.0012, val_loss = 0.0011
>>>> @ 10, training_loss = 0.0010, val_loss = 0.0009
>>>> @ 11, training_loss = 0.0009, val_loss = 0.0009
>>>> @ 12, training_loss = 0.0008, val_loss = 0.0008
>>>> @ 13, training_loss = 0.0008, val_loss = 0.0008
>>>> @ 14, training_loss = 0.0008, val_loss = 0.0007
>>>> @ 15, training_loss = 0.0019, val_loss = 0.0017
>>>> @ 16, training_loss = 0.0014, val_loss = 0.0013
>>>> @ 17, training_loss = 0.0011, val_loss = 0.0009
>>>> @ 18, training_loss = 0.0009, val_loss = 0.0010
>>>> @ 19, training_loss = 0.0009

In [17]:
from sklearn.ensemble import IsolationForest
from sklearn import covariance
from scipy.spatial import distance
import numpy as np

# Empirical Covariance
group_lasso = covariance.EmpiricalCovariance(assume_centered=False)
group_lasso.fit(x_train_array)
incov = group_lasso.precision_
mean = np.mean(x_train_array, axis=0)
mean = list(mean)

scores = []
for i in range(len(x_test_array)):
    scores.append(distance.mahalanobis(mean, x_test_array[i], incov))
ma_scores = np.array(scores)

# Isolation Forest
iso_Forest = IsolationForest(n_estimators=100, random_state=123)
iso_Forest.fit(x_train_array)
if_scores = iso_Forest.decision_function(x_test_array)

# Autoencoder Scores (AE Scores)
x_test_recon_np = x_test_recon.detach().cpu().numpy()  # Tensörü NumPy'e çevirme
ae_scores = np.mean((x_test_array - x_test_recon_np) ** 2, axis=1)

# Diğer inputlar için işlemler
inputs = [
    [train_en_out[0].detach().cpu().numpy(), test_en_out[0].detach().cpu().numpy()],
    [train_en_out[1].detach().cpu().numpy(), test_en_out[1].detach().cpu().numpy()],
    [train_en_out[2].detach().cpu().numpy(), test_en_out[2].detach().cpu().numpy()],
    [train_en_out[3].detach().cpu().numpy(), test_en_out[3].detach().cpu().numpy()],
    [train_latant_v.detach().cpu().numpy(), test_latant_v.detach().cpu().numpy()],
    [train_de_out[0].detach().cpu().numpy(), test_de_out[0].detach().cpu().numpy()],
    [train_de_out[1].detach().cpu().numpy(), test_de_out[1].detach().cpu().numpy()],
    [train_de_out[2].detach().cpu().numpy(), test_de_out[2].detach().cpu().numpy()],
    [train_de_out[3].detach().cpu().numpy(), test_de_out[3].detach().cpu().numpy()],
    [train_de_out[4].detach().cpu().numpy(), test_de_out[4].detach().cpu().numpy()],
]

efe_scores = []
dfe_scores = []

for inp in inputs[:5]:  # EFE Scores için
    group_lasso.fit(inp[0])
    incov = group_lasso.precision_
    mean = np.mean(inp[0], axis=0)
    mean = list(mean)
    scores = []
    for i in range(len(inp[1])):
        scores.append(distance.mahalanobis(mean, inp[1][i], incov))
    efe_scores.append(np.array(scores))

for inp in inputs[5:]:  # DFE Scores için
    group_lasso.fit(inp[0])
    incov = group_lasso.precision_
    mean = np.mean(inp[0], axis=0)
    mean = list(mean)
    scores = []
    for i in range(len(inp[1])):
        scores.append(distance.mahalanobis(mean, inp[1][i], incov))
    dfe_scores.append(np.array(scores))

# Tüm skorları birleştir
scores = [ma_scores] + [if_scores] + [ae_scores] + efe_scores + dfe_scores


In [13]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score, average_precision_score
from sklearn.metrics import precision_score, recall_score, f1_score

scores_df=pd.DataFrame({"ma_scores":scores[0],
                        "if_scores":scores[1],
                        "ae_scores":scores[2],
                        "efe_scores_1":scores[3],
                        "efe_scores_2":scores[4],
                        "efe_scores_3":scores[5],
                        "efe_scores_4":scores[6],
                        "efe_scores_5":scores[7],
                        "dfe_scores_1":scores[8],
                        "dfe_scores_2":scores[9],
                        "dfe_scores_3":scores[10],
                        "dfe_scores_4":scores[11],
                        "dfe_scores_5":scores[12],
                        "label":y_test})

x=scores_df.drop("label",axis=1)
x=scores_df[["ma_scores"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("ma_scores") #only mahal of input
print(test_acc)
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["if_scores"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("if_scores") #isolation forest
print(test_acc)
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["ae_scores"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("ae_scores") # use recon loss
print(test_acc)
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["efe_scores_1","efe_scores_2","efe_scores_3","efe_scores_4","efe_scores_5"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("efe_scores") # use encoder mahal 
print(test_acc)
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["dfe_scores_1","dfe_scores_2","dfe_scores_3","dfe_scores_4","dfe_scores_5"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("dfe_scores") # use decoder mahal
print(test_acc) 
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["ma_scores","efe_scores_1","efe_scores_2","efe_scores_3","efe_scores_4","efe_scores_5"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("mahal+efe_scores") # use mahal for input and encoder mahal
print(test_acc) 
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["ma_scores","dfe_scores_1","dfe_scores_2","dfe_scores_3","dfe_scores_4","dfe_scores_5"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("mahal+dfe_scores") # use mahal for input and decoder mahal
print(test_acc) 
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["ma_scores","ae_scores"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("mahal+ae_scores") # use mahal for input and recon loss
print(test_acc)
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["efe_scores_1","efe_scores_2","efe_scores_3","efe_scores_4","efe_scores_5","dfe_scores_1","dfe_scores_2","dfe_scores_3","dfe_scores_4","dfe_scores_5"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("efe_scores+dfe_scores") # use encoder mahal and decoder mahal
print(test_acc)
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["ae_scores","efe_scores_1","efe_scores_2","efe_scores_3","efe_scores_4","efe_scores_5"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("ae_scores+efe_scores") # use encoder mahal and recon loss
print(test_acc)
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["ae_scores","dfe_scores_1","dfe_scores_2","dfe_scores_3","dfe_scores_4","dfe_scores_5"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("ae_scores+dfe_scores") # use decoder mahal and recon loss
print(test_acc)
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["ma_scores","efe_scores_1","efe_scores_2","efe_scores_3","efe_scores_4","efe_scores_5","dfe_scores_1","dfe_scores_2","dfe_scores_3","dfe_scores_4","dfe_scores_5"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("ma_scores+efe_scores+dfe_scores") # # use mahal for input, encoder mahal and decoder mahal
print(test_acc)
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["ma_scores","ae_scores","efe_scores_1","efe_scores_2","efe_scores_3","efe_scores_4","efe_scores_5"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("ma_scores+ae_scores+efe_scores") # # use mahal for input, recon loss, encoder mahal
print(test_acc)
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["ma_scores","ae_scores","dfe_scores_1","dfe_scores_2","dfe_scores_3","dfe_scores_4","dfe_scores_5"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("ma_scores+ae_scores+dfe_scores") # # use mahal for input, recon loss, decoder mahal
print(test_acc)
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["ae_scores","efe_scores_1","efe_scores_2","efe_scores_3","efe_scores_4","efe_scores_5","dfe_scores_1","dfe_scores_2","dfe_scores_3","dfe_scores_4","dfe_scores_5"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("ae_scores+efe_scores+dfe_scores") # # use recon loss, encoder mahal, and decoder mahal
print(test_acc)
print(precision)
print(recall)
print(AUROC)
print(AUPRC)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

x=scores_df.drop("label",axis=1)
x=scores_df[["ma_scores","ae_scores","efe_scores_1","efe_scores_2","efe_scores_3","efe_scores_4","efe_scores_5","dfe_scores_1","dfe_scores_2","dfe_scores_3","dfe_scores_4","dfe_scores_5"]]
y=scores_df["label"].values
clf = LogisticRegression(random_state=0).fit(x, y)
model=clf
test_acc = model.score(x, y)
AUROC = roc_auc_score(y,model.predict_proba(x)[:,1])
AUPRC = average_precision_score(y,model.predict_proba(x)[:,1])
recall = recall_score(y,  model.predict(x))
precision = precision_score(y, model.predict(x))
f1_score_=f1_score(y_true=y, y_pred =model.predict(x))
fpr, tpr, thresholds = roc_curve(y,model.predict_proba(x)[:,1])
tn, fp, fn, tp = confusion_matrix(scores_df["label"].values, model.predict(x)).ravel()

print("ma_scores+ae_scores+efe_scores+dfe_scores") # # use mahal for input, recon loss, encoder mahal, and decoder mahal
print(test_acc)
print(AUROC)
print(AUPRC)
print(precision)
print(recall)
print(f1_score_)
print(fp/(fp+tn))
print("\n")

ma_scores
0.8999
0.9347684279191129
0.8598
0.9806419599999999
0.9727128447978912
0.895718303989999
0.06


if_scores
0.9345
0.9214354995150339
0.95
0.9597928
0.9065252018995851
0.9354997538158543
0.081


ae_scores
0.8155
0.9349324510614834
0.6782
0.9707798000000001
0.963063082187282
0.7861365480468298
0.0472


efe_scores
0.9274
0.9508438818565401
0.9014
0.98504532
0.9834315933085407
0.9254620123203285
0.0466


dfe_scores
0.8647
0.9309855826045852
0.7878
0.92960526
0.9176412064254541
0.8534286642833929
0.0584


mahal+efe_scores
0.9262
0.94957805907173
0.9002
0.9853526800000002
0.983724558747253
0.9242299794661191
0.0478


mahal+dfe_scores
0.8965
0.938121546961326
0.849
0.9811413199999999
0.9741051213393801
0.8913385826771654
0.056


mahal+ae_scores
0.9002
0.935
0.8602
0.9806946799999998
0.9727890360137151
0.8960416666666666
0.0598


efe_scores+dfe_scores
0.9285
0.9530556142947769
0.9014
0.9853533200000001
0.9842253604546489
0.9265083770171652
0.0444


ae_scores+efe_scores
0.9272
0.950632